<a id =6></a><br>
The project is outlined as following:
 
 - [THE DATA](#1)
 
 - [ADDING NEW COLUMNS](#2)
 
 - [GATHERING EVENTS](#3)
 
 - [FINDING TRAJECTORIES](#4)
 
 - [VISUALIZING THE EVENTS](#5)
 - [DETECTOR MATRIX](#7)
 
 

# <center><a id =1></a><br> THE DATA: Calibration run 260 / Real Measurement run 333


In this project, we analyze the data obtained through a set of particle detectors. The detectors consist on four layers of cells fulls of a gas. Each of the cells is traversed by an anodic cable in its middle. When a particle traverses the gas, it ionizes it, with the ions being then attracted towards the cable. The cell then emmits a signal when the ions reach the cable. \
The ions can be considered to travel at a constat drift velocity $v_d$ towards the cable. The impact possition in a cell $x$ can be expressed as
    
    
$$
x = v_d (t - t_0),
$$
    
being $t_0$ a pedestal time and $t$ the time recorded by the detector. As the maximum drift time  $T_{Max} = \frac{L}{2 v_d} = 390$ ns ($L=42$ mm is the lenght of the cell) is a known country, t0 can be calculated. This will be explained later.    
\
The data obtained are encoded in a CSV file. The way we can interpret its content is exlained as it is needed. It is the goal of this project to analyze these data so that we can track the trajectories of the particles through them. More specifically, the tasks to do are:
    
    -Grouping the data into events corresponding to a particle hitting the detector
    -Obtain and plot the drift times
    
This was done for both a calibration run and for a the data obtained in a real experiment.



In [ ]:
# Important imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from os import listdir
from os.path import isfile, join, getsize
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string)) # for markdown purposes
from numba import vectorize, int64, int32, jit, njit, cuda, float64

***

# <a id =1></a><br> THE DATA: Calibration run 260 / Real Measurement run 333
[Content Outline](#6)

In [ ]:
## Set as FALSE for Calibration run 
REAL_DATA = True

# We limit the amount of data we are accepting 
if REAL_DATA:
    mypath = '../data/Run000333/'
else:
    mypath = '../data/Run000260/'

max_mb = 65 # maximum size of data in mb to read in
files = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
mb = 0
for i in range(len(files)):
    mb += getsize(files[i])/1024**2 # getsize returns size in bytes
    if mb >= max_mb:
        files = files[:(i-1)]
        break

# Load the data
data = pd.concat([pd.read_csv(file) for file in files], axis = 0)
data.reset_index(inplace=True, drop=True) #This resets the index from 0 to onwards
# data = pd.read_csv(files[0])

#Let's drop the column named 'HEAD'
data.drop('HEAD', inplace=True, axis=1) # We remove the 'HEAD', inplace = True because we return nothing

# We print the description
printmd('## DESCRIPTION OF THE DATA')
print('Used files:', '\n', files, '\n') # To check which files are read
print(data.info())
display(data.describe())

In [ ]:
#Visualization of the first 5 entries of the file
printmd('## THE DATAFRAME')
printmd('The dataframe below presents the first 10 hits as presented in the raw data')
display(data.head(10))

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,4))

df_0 = data[data['FPGA']==0]
df_1 = data[data['FPGA']==1]

max_channel = data['TDC_CHANNEL'].max()
df_0.hist('TDC_CHANNEL', ax=ax1, bins=max_channel)#, range=(137,138))
df_1.hist('TDC_CHANNEL', ax=ax2, bins=max_channel)#, range=(137,138))
ax1.set_title('FPGA 0')
ax2.set_title('FPGA 1')

printmd('## HISTOGRAM OF HITS PER CHANNEL')
plt.show()
printmd('#### There are a lot of hits falling under the mean time trigger')
# Remark: There is a lot of data in the trigger

***
# <a id =2></a><br>  ADDING NEW COLUMNS: TIME, DETECTOR, LAYER, AND CELL
 [Content Outline](#6)

## CONSTANTS: TRIGGER CHANNELS AND TMAX, L, Vd

In [ ]:
st1 = 137  # Scintillator trigger channel 1
st2 = 138  # Scintillator trigger channel 2
mtt = 139  # Mean-time trigger channel
T_M = 390  #(maximum drift time in ns)
L = 42  # (lenght of the cell in mm)
v_d = L/(2*T_M)  # (constant drift velocity in mm/ns)

# We set the detector column to display -1 for trigger and detector number
data['detector'] = -1

 ## TIME COLUMN
 
 Let's add a column to the existing data.

This new column will be the time measurements(ns). The time measurement in nanoseconds is given by:

```python
 data['time']=data['ORBIT_CNT']*2564*25+data['BX_COUNTER']*25+data['TDC_MEAS']*25/30
```
 
Now, the rate of passage of particles through the detector was rather limited, i.e. the chance to get two particle within the same orbit is extremely small. Therefore we can drop the orbit part of the formula yields

In [ ]:
data['time'] = data['BX_COUNTER']*25 + data['TDC_MEAS']*25/30

# We have added a new time column to the data. The first 5 rows are
data.head(5)

The actual time taken by the ionization to reach the anodic wire (drift time) is proportional to the distance:

$$
x = v_d t_i= v_d(t_R-t_0)
$$

where $v_d$ is the constant drift velocity, $t_R$ is the time recorded by the electronics and $t_0$ is a time pedestal which needs to be determined for every particles crossing the detector (the drift time is $t_R-t_0$). This is possible thanks to the geometry of the layers, more precisely by the fact that they are staggered by exactly half a cell. 

To compute the constant $t_0$, which is different for every event, we can use the following relation:

$$T_{MAX}=\frac{t_{1}+t_{3}}{2}+t_{2}$$

where $t_1 = t_{1'}-t_0$, $t_2 = t_{2'}-t_0$ and $t_3 = t_{3'}-t_0$, are the drift times of those three cells and $t_{i'}$ is the time measurement of the respective cell. The drift times can be calculated in this way because $t_0$ is essentially the same for each cell in one event, because the particle velocity of the muons(?) is much higher than the drift velocity of the electrons. Then the relation becomes:

$$T_{MAX}=\frac{t_{1'}-t_0+t_{3'}-t_0}{2}+t_{2'}-t_0$$

from which we get:

$$t_0=\frac{t_{1'}+t_{3'}+2t_{2'}-2T_{MAX}}{4}$$

Finally we notice that $t_{1'}$, $t_{2'}$, $t_{3'}$ are the times recorded by each cell, which are already available in our dataset.

## DETECTOR, LAYER, AND CELL COLUMNS

Before processing the dataset, we have to create some missing columns, in fact the DataFrame with the events must contain the following information:
* CHAMBER, which is the Detector number. The data to be analyzed have been gathered by a series of (4) particle detectors: [1-4]
* LAYER.  Such detectors were composed by four layers of cells: [1-4];
* CELL, which is in the number of the cell.  The actual detector has 16 cells per layer: [1-16]
* POSTION, which is the position where a particle traverses the cell [0-21] (in mm. The maximum 21 mm value corresponds to the position of the anodic wire to where the particles travel. In this case the left-right ambiguity is NOT resolved). 



![(Picture of detector configuration)](./Detector.png)

The channels are mapped to the four detectors in the following ways:

* Detector 1 $\rightarrow$ FPGA 0, TDC_CHANNEL in [1-64]
* Detector 2 $\rightarrow$ FPGA 0, TDC_CHANNEL in [65-128]
* Detector 3 $\rightarrow$ FPGA 1, TDC_CHANNEL in [1-64]
* Detector 4 $\rightarrow$ FPGA 1, TDC_CHANNEL in [65-128]

Notice that since we have 4 layers with 16 cells per layes, each detector will have 64 cells. Therefore, we can assume that the TDC_CHANNEL is a label for all the cells in the chamber.

Even though not used as such, an external trigger was present anyway. When it fired a special 64 bit word (a row in the csv file) were produced enconding the trigger information, i.e. the timing at which it occurred and which of the kind of trigger fired. More precisely:

* Mean-time trigger: TDC_CHANNEL=139
* Scintillator trigger: TDC_CHANNEL=137 or 138

In [ ]:
# adding detector, layer and cell columns 

@vectorize([int64(int64, int64)])
def giveDetector(fpga, channel):
    if channel > 128:
        return -1

    channel -= 1
    return fpga*2 + 1 + channel // 64

@vectorize([int64(int64)])
def giveLayer(channel):
    if channel > 128:
        return -1

    layer_map = {0 : 1,
                 1 : 4,
                 2 : 2,
                 3 : 3}

    cell_index = channel % 64
    layer = layer_map[cell_index % 4]
    
    return layer

@vectorize([int64(int64)])
def giveCell(channel):
    if channel > 128:
        return -1

    channel -= 1
    cell_index = channel % 64
    cell = cell_index // 4 + 1

    return cell

# Test of functions
if __name__ == '__main__':
    first_cell_of_layer = 67
    a = np.arange(first_cell_of_layer,first_cell_of_layer+16*4,4)
    print(list(zip(giveDetector(0,a), giveLayer(a), giveCell(a))))

In [ ]:
# Calculation of detector, layer & cell according to the previous functions
data['detector'] = giveDetector(data['FPGA'], data['TDC_CHANNEL'])
data['layer'] = giveLayer(data['TDC_CHANNEL'])
data['cell'] = giveCell(data['TDC_CHANNEL'])

display(data.head(5))

***
# <a id =3></a><br> FINDING EVENTS 
 [Content Outline](#6)

A trigger dataset is formed by applying a mask to the original data in order to isolate the hits that correspond to the TDC_CHANNEL 139. Similar lines of code are presented for the two other trigger channels, scintillator 1 and 2. 

In [ ]:
trigger_mtt = data.loc[data['TDC_CHANNEL']==mtt, ['ORBIT_CNT','time']]
trigger_st = data.loc[(data['TDC_CHANNEL']==st1) | (data['TDC_CHANNEL']==st2), ['ORBIT_CNT','time']]

display(trigger_mtt.head())
if not trigger_st.empty:
    display(trigger_st.head())

Using the .isin() function, the orbits found in trigger_mtt data is isolated from the original dataset to produce the 'events' dataset because, supposedly, **one event corresponds to one common orbit.** The next lines of code sorts the by orbits and reindexed to present the data for each event.

In [ ]:
#.isin function compares the two arrays and takes only the orbits that are found in the trigger
events_mtt = data.loc[data['ORBIT_CNT'].isin(trigger_mtt['ORBIT_CNT'])]
events_st = data.loc[data['ORBIT_CNT'].isin(trigger_st['ORBIT_CNT'])]

def create_events(df):
    events = df.copy(deep=False)
    # We remove the events that are in channels above 128
    events = events[events['TDC_CHANNEL'] <= 128]

    # We now group by orbit
    events['event'] = events.groupby('ORBIT_CNT').ngroup()+1

    # Common orbits correspond to a single event
    events.set_index(['event', events.index], inplace=True) # We make the events column an index
    events.rename_axis(('event','index'), inplace=True)
    events.sort_values(['ORBIT_CNT','detector','layer','cell'], inplace=True) # We sort according to these categories

    printmd('## EVENTS DATAFRAME')
    display(events.head(4))
    
    return events

events_mtt = create_events(events_mtt)
display(events_mtt.head(5))
if not trigger_st.empty:
    events_st = create_events(events_st)
    display(events_st.head(5))

***
# <a id =4></a><br> FINDING THE TRAJECTORIES, computing t0
 [Content Outline](#6)

In [ ]:
def findTrajectories(df):
    data = df.copy(deep=False)
    data['t0_1'] = 0
    data['t0_2'] = 0
    for i in data.index.get_level_values(0).unique():
        present_detectors = data.loc[i,'detector'].unique()
        ev = data.loc[i].copy(deep=False)
        
        for det in present_detectors:
            sub_ev = ev.loc[ev['detector'] == det].copy(deep=False)
            for l in range(1,4):
                fst_layer = sub_ev.loc[sub_ev['layer'] == l, 'cell']
                sec_layer = sub_ev.loc[sub_ev['layer'] == l+1, 'cell']

                # Find successor for each cell in layer
                if l % 2 == 0:
                    successor_mask = fst_layer.isin(sec_layer) | fst_layer.isin(sec_layer+1)
                else:
                    successor_mask = fst_layer.isin(sec_layer) | fst_layer.isin(sec_layer-1)
                
                # Drop whole sub_event if the current layer is completely empty
                if successor_mask.sum() == 0:
                    sub_ev['t0_1'] = np.nan
                    sub_ev['t0_2'] = np.nan
                    break

                else:
                    # Drop rows where no successor was found
                    sub_ev.loc[(successor_mask==False).index, 't0_1'] = np.nan
                    sub_ev.loc[(successor_mask==False).index, 't0_2'] = np.nan

            na_mask = sub_ev['t0_1'].notna()
            if na_mask.sum() > 0:
                sub_ev.loc[na_mask,'t0_1'] = 0.5*(0.5*(sub_ev.loc[(sub_ev.loc[na_mask, 'layer']==1).index, 't0_1'].iloc[0]+
                                                        sub_ev.loc[(sub_ev.loc[na_mask, 'layer']==3).index, 't0_1'].iloc[0])+
                                                  sub_ev.loc[(sub_ev.loc[na_mask, 'layer']==2).index, 't0_1'].iloc[0])
                sub_ev.loc[na_mask, 't0_2'] = 0.5*(0.5*(sub_ev.loc[(sub_ev.loc[na_mask, 'layer']==2).index, 't0_1'].iloc[0]+
                                                        sub_ev.loc[(sub_ev.loc[na_mask, 'layer']==4).index, 't0_1'].iloc[0])+
                                                   sub_ev.loc[(sub_ev.loc[na_mask, 'layer']==3).index, 't0_1'].iloc[0])

    return data
            

# np.isin or in function of numpy/python is not supported by numba
# That's why we make a new .isin function to be able to use the @njit decorator
@njit
def myIsin(array_a, array_b):
    result = np.zeros(len(array_a), dtype=np.bool_)
    for i,a in enumerate(array_a):
        for j,b in enumerate(array_b):
            if a == b:
                result[i] = True
                break
    return result


@njit
def findTrajectories_np(data, inds, columns): # data, index, columns??
    det_ind, lay_ind, cel_ind, time_ind, t0_1_ind, t0_2_ind = columns #detector, layer, cel, time, 1t0, 2t0 are the columns # use old version???
    #np.roll (array, shift, axis)  #elements of the input array are being shifted
    ev_locs = np.where(inds - np.roll(inds,1)!=0)[0]  #np.where(condition,[x,y]) return elements chosen from x or y depending on condition.
    ev_locs = np.append(ev_locs, [len(inds)])

    for i in range(len(ev_locs)-1):
    
        ev = data[ev_locs[i]:ev_locs[i+1]]

        # Every row registered in same detector?
        if np.unique(ev[:,det_ind]).size != 1:
            sub_ev_locs = np.where(ev[:,det_ind]-np.roll(ev[:,det_ind],1)!=0)[0]
            sub_ev_locs = np.append(sub_ev_locs, [len(ev[:,det_ind])])

        else:
            sub_ev_locs = np.array([0,len(ev[:,det_ind])])

        for j in range(len(sub_ev_locs)-1):
            sub_ev = ev[sub_ev_locs[j]:sub_ev_locs[j+1]]

            for l in range(1,4):
                fst_layer = sub_ev[sub_ev[:,lay_ind]==l,cel_ind]
                sec_layer = sub_ev[sub_ev[:,lay_ind]==l+1,cel_ind]

                # Find successor for each cell in layer
                if l % 2 == 0:
                    successor_mask = myIsin(fst_layer, sec_layer) | myIsin(fst_layer, sec_layer+1)
                else:
                    successor_mask = myIsin(fst_layer, sec_layer) | myIsin(fst_layer, sec_layer-1)

                # Drop whole sub_event if the current layer is completely empty
                if np.any(successor_mask) == False:
                    sub_ev[:,t0_1_ind] = np.nan
                    sub_ev[:,t0_2_ind] = np.nan
                    break

                # Else mark rows where no successor was found
                # if one cell was found with no successor
                elif np.all(successor_mask) == False: # False has to be np.array([False]) so that numba broadcasts
                    idx = np.nonzero(successor_mask==np.array([False]))[0] + np.nonzero(sub_ev[:,lay_ind]==l)[0][0]
                    sub_ev[idx,t0_1_ind] = np.nan
                    sub_ev[idx,t0_2_ind] = np.nan

            # Compute t0 if a trajectory was found
            mask = ~np.isnan(sub_ev[:,t0_1_ind]) # mask where values are numbers and not NaN
            if np.any(mask) == True:
                sub_ev[mask,t0_1_ind] = 0.5*(0.5*(sub_ev[sub_ev[mask,lay_ind]==1,time_ind][0]+
                                           sub_ev[sub_ev[mask,lay_ind]==3,time_ind][0])+
                                      sub_ev[sub_ev[mask,lay_ind]==2,time_ind][0]-T_M)
                sub_ev[mask,t0_2_ind] = 0.5*(0.5*(sub_ev[sub_ev[mask,lay_ind]==2,time_ind][0]+
                                           sub_ev[sub_ev[mask,lay_ind]==4,time_ind][0])+
                                      sub_ev[sub_ev[mask,lay_ind]==3,time_ind][0]-T_M)

    return data

In [ ]:
def calculate_t0(df, numba_optimized=True):
    events = df.copy(deep=False)

    if not numba_optimized:
        events = findTrajectories(events)

    else:
        events['t0_123'] = 0
        events['t0_234'] = 0
        level0_inds = events.index.get_level_values(0).to_numpy()
        c_names = ['detector', 'layer', 'cell', 'time', 't0_123', 't0_234']
        columns = np.array([events.columns.get_loc(column) for column in c_names])

        data = findTrajectories_np(events.to_numpy(), level0_inds, columns)
        events = pd.DataFrame(data=data, index=events.index, columns=events.columns)

        events['detector'] = events['detector'].astype(np.int32)
        events['layer'] = events['layer'].astype(np.int32)
        events['cell'] = events['cell'].astype(np.int32)

    return events

events_mtt = calculate_t0(events_mtt)
display(events_mtt.head(5))
if not trigger_st.empty:
    events_st = calculate_t0(events_st)
    display(events_st.head(5))

***
# <a id =5></a><br> 'VISUALIZING' THE EVENTS
 [Content Outline](#6)
## <span style='color:  gray'> CALCULATION OF DRIFT TIMES </span> 

In [ ]:
def make_hist(df):
    events = df.copy(deep=False)

    events['drift_time'] = events['time'] - events['t0_123'] # use one t0???
    events['x'] = v_d*events['drift_time']
    # Clean up data for events which have a negative drift time or a position over 21mm
    events.loc[(events['drift_time']<0) | (events['x']>L/2), 't0_123'] = np.nan

    events.loc[events['t0_123'].notna(),'drift_time'].plot(kind='hist', title='Distribution of Drift Times',
                             bins=100) # driftime instead of x???

    plt.xlabel('Drift time (ns)')
    plt.ylabel('Number of Occurrences')

    plt.show()

    printmd('#### Remark: the maximum amount of occurences happen around 10 mm from the anodic wire.')
    
    return events


display(events_mtt.head(5))
events_mtt = make_hist(events_mtt)
if not trigger_st.empty:
    events_st = make_hist(events_st)
    display(events_st.head(5))

## <span style='color:  gray'> FILTERING THE DATA </span>

###  Useful events: showing the percentage of events that have more than 3 hits?

In [ ]:
def print_useful_events(df):
    events = df.copy(deep=False)

    # Ratio of "useful events" to all events
    try:
        print('"useful rows" / "all rows": ', 1 - len(events[events['t0_123'].isna()])/events.shape[0])
    except ZeroDivisionError:
        print('Error: Dataframe is empty')

    # Event is counted as useful if 4 or more times are available
    event_groups = events.loc[~events['t0_123'].isna(),'time'].groupby('event').count()
    # print(event_groups) 
    try:
        print('"useful events" / "all events"', (event_groups>3).sum()/len(event_groups))  # some events only have 3 (or below) hits
    except ZeroDivisionError:
        print('Error: Dataframe is empty')


print('Mean Time Trigger:')
print_useful_events(events_mtt)
if not trigger_st.empty:
    print('Scintillator Trigger')
    print_useful_events(events_st)

### Filter 2: Drop hits that do not align
### Filter 3: Obtain events that have at least one hit per layer (i.e. 4 layer representation)
### Filter 4: Sort OUT events that have only ONE detector representation

In [ ]:
def prepare_for_vis(df):
    # Drop all not aligning hits
    visual_evs = df.dropna(subset=['t0_123']).copy() # subset = array which limits the dropping process to passed rows/columns through list.

    # Sort out events which are not in every layer, each event must have 4 layers
    go_through_each_lay = visual_evs.loc[:,'layer'].groupby(level=0).nunique() == 4

    # Sort out events which are only registered in one detector
    go_through_each_lay = (visual_evs.loc[:,['layer','detector']].groupby(['event','detector']).nunique() == 4).loc[:,'layer']
    go_through_each_lay = go_through_each_lay.groupby('event').all()
    # Sort out events which are only registered in one detector
    multiple_detectors = visual_evs.loc[:,'detector'].groupby('event').nunique() > 1
    # Combine the two masks
    index_mask = go_through_each_lay & multiple_detectors

    # Apply it to dataframe
    visual_evs = visual_evs.loc[visual_evs.index.get_level_values(0).map(index_mask),:]

    # Reindex the event index level starting from 1
    visual_evs.reset_index(level='event', inplace=True)
    visual_evs.loc[:,'event'] = (visual_evs.loc[:,'event'].diff()!=0).cumsum()
    visual_evs.set_index(['event', visual_evs.index], inplace=True)
    
    return visual_evs


events_mtt_vis = prepare_for_vis(events_mtt)
display(events_mtt_vis.head(4))
if not trigger_st.empty:
    events_st_vis = prepare_for_vis(events_st)
    display(events_st_vis.head(4))

##  <a id =7></a><br> <span style='color:  gray'> MAKING THE DETECTOR MATRIX </span>
 [Content Outline](#6)
 

In [ ]:
printmd("### Event Data")
def plotDetector(ax, event, is_empty):
    detector = np.zeros((4,16*2+1))

    even_layers = event['layer']%2 == 0
    odd_layers = even_layers == False

    def fillDetectorMatrix(matrix, i, j, s=0):
        matrix[i,j*2+s] = 1
        matrix[i,j*2+1+s] = 1

    def plotLeftRight(ax, cell, position, layer, odd, left):
        if left == True:
            ax.scatter((cell-1-position/L)*2+0.5+odd, layer-1, s=10, c='g', marker='+')
        else:
            ax.scatter((cell-1+position/L)*2+0.5+odd, layer-1, s=10, c='b', marker='+')
    
    if is_empty == False:
        fillDetectorMatrix(detector, event.loc[even_layers,'layer']-1,
                    event.loc[even_layers,'cell']-1, s=0)
        fillDetectorMatrix(detector, event.loc[odd_layers,'layer']-1,
                    event.loc[odd_layers,'cell']-1, s=1)

        for odd,left in [[odd, left] for odd in [True, False] for left in [True,False]]:
            if odd == True:
                layers = odd_layers
            else:
                layers = even_layers

            plotLeftRight(ax, event.loc[layers, 'cell'], event.loc[layers, 'x'],
                     event.loc[layers, 'layer'], odd, left)

    detector[(0,1,2,3),(0,32,0,32)] = 0.5 # Mark "cell offset"
    ax.matshow(detector, origin='lower', cmap='bone', vmin=0, vmax=1)

    # Major ticks
    ax.set_xticks(np.arange(1.5, 32, 2));
    ax.set_yticks(np.arange(0, 4, 1));

    # # Labels for major ticks
    ax.set_xticklabels(np.arange(1, 17, 1));
    ax.set_yticklabels(np.arange(1, 5, 1));

    # # Minor ticks
    # # ax.set_xticks(np.arange(-.5, 32, 2), minor=True);
    ax.set_yticks(np.arange(0.5, 4, 1), minor=True);

    ax.grid(which='minor', axis='y', color='w', linewidth=1)
    ax.xaxis.set_ticks_position('bottom')
    
    vert_line = np.array([0,1])
    for l in range(4):
        for c in range(16):
            if l%2 == 0:
                ax.plot([c*2+0.5,c*2+0.5], vert_line+l-0.5, c='w')
            else:
                ax.plot([c*2+1.5,c*2+1.5], vert_line+l-0.5, c='w')

def plotEvent(event):
    fig, axes = plt.subplots(2, 2, figsize=(12,4))

    for i in range(0,4):
        sub_ev = event.loc[event['detector']==i+1, :]
        plotDetector(axes[1-i%2,i//2], sub_ev, sub_ev.empty)
        axes[1-i%2,i//2].set_title('Detector {}'.format(i+1))
    # TODO legend

    plt.show()


printmd('Blue dots indicate paths at the right of the anodic wire, Green dots indicate paths at the left as in the project description')
printmd("### Event Path 1")
plotEvent(events_mtt_vis.loc[47])

printmd("### Event Path 2")
plotEvent(events_mtt_vis.loc[500])

printmd("### Event Path 3")
plotEvent(events_mtt_vis.loc[450])